## **Cassava VGG training**
2021/12/09 written by T.Yonezu

In [11]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader, Dataset

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import glob 
import os
from tqdm import tqdm

from cassava_dataset import *
from myvgg import *

import warnings
warnings.simplefilter('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
input_dir = os.path.join('..',"..", 'input', 'cassava-leaf-disease-classification')

In [13]:
x = pd.read_csv(os.path.join(input_dir, 'train.csv'))
x["image_path"] = os.path.join(input_dir,"train_images")
x["image_path"] = x["image_path"].str.cat(x["image_id"], sep=os.path.sep)

train_dict = dict( zip(x["image_path"],x["label"]) )

In [14]:
size = (224,224)
mean = [0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

transform = ImageTransform(size,mean,std)

In [15]:
BATCH_SIZE = 5
NUM_WORKERS = 8

train_data = CassavaDataset(train_dict,transform=transform)
train_data = DataLoader(train_data,batch_size=BATCH_SIZE)

In [20]:
EPOCH_NUM = 3

OUT_DIR = os.path.join("..","..","input","cassava-models")
MODEL_NAME = "VGG_cassava_trained_%dEpoch.mdl"
PATH = os.path.join(OUT_DIR,MODEL_NAME%EPOCH_NUM)

In [18]:
import torch.optim as optim
from torch import nn

model = VGG11(num_classes=5)
model = model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01)
criterioEPOCH_NUMn.CrossEntropyLoss()


# in your training loop:
for epoch in tqdm(range(EPOCH_NUM)):
    for batch in (train_data):
        
        X = batch[0].cuda()
        y = batch[1].cuda()

        pred = model(X)
        
        # zero the gradient buffers
        optimizer.zero_grad()
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step() # Does the update
        

if not(os.path.exists(OUT_DIR)):
    os.makedirs(OUT_DIR)
torch.save(model.state_dict(),PATH)

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [09:36<00:00, 192.28s/it]


In [21]:
model = VGG11(num_classes=5)
model.load_state_dict(torch.load(PATH))
model.eval()

VGG11(
  (block1_output): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2_output): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3_output): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block4_output): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3)